# K-nn by Daphne Annink


## import the data
Here i import the data like given in the assignment \
When importing the validation data change the date's so it sets the lables for 2001 corectly

In [ ]:
import numpy as np

data = np.genfromtxt('dataset1.csv', delimiter=';', usecols=[1,2,3,4,5,6,7], converters={5: lambda s: 0 if s == b"-1" else float(s), 7: lambda s: 0 if s == b"-1" else float(s)})
dates = np.genfromtxt('dataset1.csv', delimiter=';', usecols=[0])
labels = []
for label in dates:
  if label < 20000301:
    labels.append('winter')
  elif 20000301 <= label < 20000601:
    labels.append('lente')
  elif 20000601 <= label < 20000901:
    labels.append('zomer')
  elif 20000901 <= label < 20001201:
    labels.append('herfst')
  else:
    labels.append('winter')
labels = np.array(labels)


validation_data = np.genfromtxt('validation1.csv', delimiter=';', usecols=[1,2,3,4,5,6,7], converters={5: lambda s: 0 if s == b"-1" else float(s), 7: lambda s: 0 if s == b"-1" else float(s)})
validation_dates = np.genfromtxt('validation1.csv', delimiter=';', usecols=[0])
validation_labels = []
for label in validation_dates:
  if label < 20010301:
    validation_labels.append('winter')
  elif 20010301 <= label < 20010601:
    validation_labels.append('lente')
  elif 20010601 <= label < 20010901:
    validation_labels.append('zomer')
  elif 20010901 <= label < 20011201:
    validation_labels.append('herfst')
  else:
    validation_labels.append('winter')
validation_labels = np.array(validation_labels)

## normalize data

I made use of the sklearn library and use there normilize function,\
The  preprocessing.normalize is standart defined to normilize over each sample so i changed the axis to 0 witch wil then normilize over each feature.\
You can see that for example in the data[1] & data[4] the first feature, it has 37 and 38 as values witch get normilzed to a closely 0.04702635 and 0.04829734

In [303]:
from sklearn import preprocessing

print(data[0])
print(data[1])
print(data[2])
print(data[3])

normilized_data = preprocessing.normalize(data, axis=0)
normilized_validation_data = preprocessing.normalize(validation_data, axis=0)

print(normilized_data[0])
print(normilized_data[1])
print(normilized_data[2])
print(normilized_data[3])


[25. 61. 35. 81.  0. 20. 10.]
[37. 73. 54. 87.  0.  0.  0.]
[61. 84. 64. 96.  0. 64. 45.]
[ 38.  64.  43.  94.   0.  82. 109.]
[0.03177456 0.02617682 0.02158506 0.02632969 0.         0.03142608
 0.01081499]
[0.04702635 0.03132636 0.03330266 0.02828004 0.         0.
 0.        ]
[0.07752994 0.03604677 0.03946982 0.03120556 0.         0.10056344
 0.04866745]
[0.04829734 0.02746421 0.02651878 0.03055544 0.         0.12884691
 0.11788338]


## K-nn functions

Here i worked out the K-nn function like given in the reader and a function to check the results of the K-nn workings \
A big difference in the K-nn algorythm is how you calculate the distance, this is why i made a seperate calculation function where i can test the results of differen distance mesurements

In [304]:
from collections import Counter
import random


def calculate_distance(training_data, classifying_point, metric="euclidean"):
    """
    Calculates distances based on the chosen metric.

    Parameters:
    - training_data: numpy array of shape (n_samples, n_features).
    - classifying_point: numpy array of shape (n_features,).
    - metric: str, distance metric to use ("euclidean", "manhattan", "minkowski", "cosine", "hamming").

    Returns:
    - distances: numpy array of shape (n_samples,), distances to the classifying_point.
    """
    if metric == "euclidean": # usefull forStraightforward / continuous data
        distances = np.linalg.norm(training_data - classifying_point, axis=1)
    elif metric == "manhattan": # Grid-like data or reduce influence of "uitschieters"
        distances = np.sum(np.abs(training_data - classifying_point), axis=1)
    elif metric == "cosine": # handy for higher dimentions or sparse data
        dot_product = np.dot(training_data, classifying_point)
        norm_a = np.linalg.norm(training_data, axis=1)
        norm_b = np.linalg.norm(classifying_point)
        distances = 1 - (dot_product / (norm_a * norm_b))
    elif metric == "mahalanobis": # looks more at relations between features
        cov_matrix = np.cov(training_data, rowvar=False)
        inv_cov_matrix = np.linalg.inv(cov_matrix)
        diff = training_data - classifying_point
        distances = np.sqrt(np.sum(np.dot(diff, inv_cov_matrix) * diff, axis=1))
    else:
        raise ValueError(f"Unknown distance metric: {metric}")
    return distances


def k_nearest_neighbours(training_data, training_labels, classifying_point, k, metric="euclidean"):
    """
    k-Nearest Neighbours Algorithm with flexible distance metrics.

    Parameters:
    - training_data: numpy array of shape (n_samples, n_features), feature vectors of training set.
    - training_labels: numpy array of shape (n_samples,), class labels of training set.
    - classifying_point: numpy array of shape (n_features,), feature vector of the test point to classify.
    - k: int, number of nearest neighbors to consider.
    - metric: str, distance metric to use ("euclidean", "manhattan", "minkowski", "cosine", "hamming").

    Return:
    - closest_point_label: label of the most frequent k closest points.
    """
    # Step 1: Calculate distances
    distances = calculate_distance(training_data, classifying_point, metric=metric)

    # Step 2: Select k closest points
    index_of_closest_points = np.argsort(distances)[:k]
    closest_points_labels = training_labels[index_of_closest_points]

    # Step 3: Determine the most frequent class
    most_common = Counter(closest_points_labels).most_common()
    tied_classes = np.array([most_common[0][0]])

    for item in most_common[1:]:
        if item[1] == most_common[0][1]:
            tied_classes = np.append(tied_classes, item[0])
        else:
            break

    # If no ties, return the result
    if len(tied_classes) == 1:
        return tied_classes[0]

    # Resolve tie by checking the closest point within the tied classes
    shortest_dist = float('inf')
    selected_class = None
    for i in range(k):
        if closest_points_labels[i] in tied_classes:
            if distances[index_of_closest_points[i]] < shortest_dist:
                shortest_dist = distances[index_of_closest_points[i]]
                selected_class = closest_points_labels[i]

    return selected_class


print(k_nearest_neighbours(normilized_data, labels, normilized_validation_data[0], 10))


    
def evaluate_k_nearest_neighbours(training_data, training_labels, validation_data, validation_labels, k, metric="euclidean"):
    """
    Evaluate k-Nearest Neighbours Algorithm results

    Parameters:
    - training_data: numpy array of shape (n_samples, n_features), feature vectors of training set.
    - training_labels: numpy array of shape (n_samples,), class labels of training set.
    - validation_data: numpy array of shape (n_samples, n_features), feature vectors of validation set.
    - validation_labels: numpy array of shape (n_samples,), class labels of validation set.
    - k: int, number of nearest neighbors to consider.
    - metric: str, distance metric to use ("euclidean", "manhattan", "minkowski", "cosine", "hamming").

    Return:
    - error procentage: the % of wrong calasifyed data form the validation set
    - predicted_labels: numpy array of shape (n_samples,), with the predicted class labels of validation set.
    """
    error_count = 0
    predicted_labels = np.array([])
    for i in range(len(validation_data)):
        predicted_label = k_nearest_neighbours(training_data, training_labels, validation_data[i], k, metric)
        predicted_labels = np.append(predicted_labels, predicted_label)
        if predicted_label != validation_labels[i]:
            error_count += 1
    return (error_count / len(validation_data)) * 100, predicted_labels


def evaluate_k_nearest_neighbours_diffrent_k(training_data, training_labels, validation_data, validation_labels, metric="euclidean"):
    """
    Evaluate k-Nearest Neighbours Algorithm results

    Parameters:
    - training_data: numpy array of shape (n_samples, n_features), feature vectors of training set.
    - training_labels: numpy array of shape (n_samples,), class labels of training set.
    - validation_data: numpy array of shape (n_samples, n_features), feature vectors of validation set.
    - validation_labels: numpy array of shape (n_samples,), class labels of validation set.
    - metric: str, distance metric to use ("euclidean", "manhattan", "minkowski", "cosine", "hamming").

    Return:
    - lowest_error: the lowest error found
    - lowest_error_k: the k that coresponds to the lowest error found
    """
    lowest_error = float("inf")
    lowest_error_k = 0
    best_predicted_lables = np.array([])
    for k in range(1, 100):
        error, predicted_labels = evaluate_k_nearest_neighbours(normilized_data, labels, normilized_validation_data, validation_labels, k, metric)
        if(lowest_error > error):
            lowest_error = error
            lowest_error_k = k
            best_predicted_lables = predicted_labels
    return lowest_error, lowest_error_k, best_predicted_lables


def log_some_examples(original_labels, given_labels, log_amount):
    """
    Logs a specified number of random examples, showing the original and predicted labels.

    Parameters:
    - original_labels: List or array of original labels.
    - given_labels: List or array of predicted labels.
    - log_amount: Number of examples to log.
    """
    random_indices = random.sample(range(len(original_labels)), log_amount)
    for idx in random_indices:
        print(f"Index {idx}: season: {original_labels[idx]} predicted season: {given_labels[idx]}")


winter


The assigmet said: "Express the error as a percentage of the validation-set that was classified incorrectly" \
this means that the lower the % the better \
\
Here i test the what the best K would be and also what the differences in different distance calculations would be \
Based on the explenation of the working of the different calculations (i noted with the calculation function itself) i am expecting cosine or mahalanobis to work the best becouse we dont have a lot of datapoints but quite a few features

In [305]:
print("Trained with euclidean:")
ecu_error, ecu_error_k, ecu_best_predicted_lables = evaluate_k_nearest_neighbours_diffrent_k(normilized_data, labels, normilized_validation_data, validation_labels, "euclidean")
print(f"lowest error: {ecu_error}, with K: {ecu_error_k}")
log_some_examples(validation_labels, ecu_best_predicted_lables, 4)

print("\nTrained with manhattan:")
manh_error, manh_error_k, manh_best_predicted_lables = evaluate_k_nearest_neighbours_diffrent_k(normilized_data, labels, normilized_validation_data, validation_labels, "manhattan")
print(f"lowest error: {manh_error}, with K: {manh_error_k}")
log_some_examples(validation_labels, manh_best_predicted_lables, 4)

print("\nTrained with cosine:")
cos_error, cos_error_k, cos_best_predicted_lables = evaluate_k_nearest_neighbours_diffrent_k(normilized_data, labels, normilized_validation_data, validation_labels, "cosine")
print(f"lowest error: {cos_error}, with K: {cos_error_k}")
log_some_examples(validation_labels, cos_best_predicted_lables, 4)

print("\nTrained with mahalanobis:")
mah_error, mah_error_k, mah_best_predicted_lables = evaluate_k_nearest_neighbours_diffrent_k(normilized_data, labels, normilized_validation_data, validation_labels, "mahalanobis")
print(f"lowest error: {mah_error}, with K: {mah_error_k}")
log_some_examples(validation_labels, mah_best_predicted_lables, 4)


Trained with euclidean:
lowest error: 50.0, with K: 54
Index 49: season: lente predicted season: zomer
Index 67: season: zomer predicted season: zomer
Index 30: season: lente predicted season: herfst
Index 48: season: lente predicted season: zomer

Trained with manhattan:
lowest error: 53.0, with K: 31
Index 61: season: zomer predicted season: zomer
Index 56: season: zomer predicted season: zomer
Index 55: season: zomer predicted season: zomer
Index 16: season: winter predicted season: herfst

Trained with cosine:
lowest error: 41.0, with K: 54
Index 42: season: lente predicted season: lente
Index 64: season: zomer predicted season: herfst
Index 85: season: herfst predicted season: zomer
Index 50: season: lente predicted season: zomer

Trained with mahalanobis:
lowest error: 42.0, with K: 79
Index 64: season: zomer predicted season: zomer
Index 52: season: zomer predicted season: lente
Index 83: season: herfst predicted season: lente
Index 86: season: herfst predicted season: zomer


Realy cool to see the predictions all togheter like this, like i expected cosine has the lowest error rate followed by mahalanobis \
i like to visualize data so i made a function to also print some random examples of the predicted labes

## The lable days exersize
Now for the final exersize, i run then with both the best calculation methods and was supprised to see that the mahalanobis only predicted "lente"\
while it did predict different oupyts in the last rounds, so i asume this is the output it suppose to be for this calculation \
\
becouse we have not a lot of input data i also tryed to take both the data an validation data combined to have more training data and predict the days again based on this, and again for both calculation methods

In [306]:
days = np.array([
    [40, 52, 2, 102, 103, 0, 0],
    [25, 48, -18, 105, 72, 6, 1],
    [23, 121, 56, 150, 25, 18, 18],
    [27, 229, 146, 308, 130, 0, 0],
    [41, 65, 27, 123, 95, 0, 0],
    [46, 162, 100, 225, 127, 0, 0],
    [23, -27, -41, -16, 0, 0, -1],
    [28, -78, -106, -39, 67, 0, 0],
    [38, 166, 131, 219, 58, 16, 41]
])
normalized_days = preprocessing.normalize(days, axis=0)

print("prediction with cosine and K 54:")
for i, day in enumerate(normalized_days):
    label = k_nearest_neighbours(normilized_data, labels, day, 54, "cosine")
    print(f'Day {i+1}: {label}')

print("\nprediction with mahalanobis and K 79:")
for i, day in enumerate(normalized_days):
    label = k_nearest_neighbours(normilized_data, labels, day, 79, "mahalanobis")
    print(f'Day {i+1}: {label}')


comib_data = np.vstack((data, validation_data))
combi_lables = np.append(labels, validation_labels)
normilized_combi_data = preprocessing.normalize(comib_data, axis=0)

print("\nprediction with combined data cosine and K 54::")
for i, day in enumerate(normalized_days):
    label = k_nearest_neighbours(normilized_combi_data, combi_lables, day, 54, "cosine")
    print(f'Day {i+1}: {label}')

print("\nprediction with combined data mahalanobis and K 79::")
for i, day in enumerate(normalized_days):
    label = k_nearest_neighbours(normilized_combi_data, combi_lables, day, 79, "mahalanobis")
    print(f'Day {i+1}: {label}')


prediction with cosine and K 54:
Day 1: winter
Day 2: lente
Day 3: herfst
Day 4: zomer
Day 5: winter
Day 6: lente
Day 7: winter
Day 8: winter
Day 9: herfst

prediction with mahalanobis and K 79:
Day 1: lente
Day 2: lente
Day 3: lente
Day 4: lente
Day 5: lente
Day 6: lente
Day 7: lente
Day 8: lente
Day 9: lente

prediction with combined data cosine and K 54::
Day 1: winter
Day 2: lente
Day 3: herfst
Day 4: zomer
Day 5: lente
Day 6: lente
Day 7: winter
Day 8: winter
Day 9: herfst

prediction with combined data mahalanobis and K 79::
Day 1: lente
Day 2: lente
Day 3: lente
Day 4: lente
Day 5: lente
Day 6: lente
Day 7: lente
Day 8: lente
Day 9: lente
